<a href="https://colab.research.google.com/github/judith182/Estadistica-y-probabilidad/blob/main/gam_other_muertes_particulas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pygam import PoissonGAM, LogisticGAM, LinearGAM
from pygam import s, f, te
from pygam.datasets import chicago, default, mcycle, faithful

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

#Chicago air pollution and death rate data
chicago(return_X_y=False)

X, y = chicago(return_X_y=True)

# y = number of death
# time = time in days
# tmpd = temperature in fahrenheit
# pm10median = median particles in 2.5-10 per cubic m
# o3median = Ozone in parts per billion
# X = [time, tmpd, pm10median, o3median]

X[:,1].min(), X[:,1].max()
X[:,3].min(), X[:,3].max()

gam = PoissonGAM(s(0, n_splines=200) + te(3, 1) + s(2)).fit(X, y)
gam.summary()

XX = gam.generate_X_grid(term=1, meshgrid=True)
Z = gam.partial_dependence(term=1, X=XX, meshgrid=True)

plt.ion()
plt.rcParams['figure.figsize'] = (12, 8)
ax = plt.axes(projection='3d')
ax.plot_surface(XX[0], XX[1], np.exp(Z), cmap='viridis')

XX = gam.generate_X_grid(term=0, meshgrid=True)
Z = gam.partial_dependence(term=0, X=XX, meshgrid=True)

## plotting
plt.plot(XX[0], np.exp(Z))

XX = gam.generate_X_grid(term=2, meshgrid=True)
Z = gam.partial_dependence(term=0, X=XX, meshgrid=True)

## plotting
plt.plot(XX[0], np.exp(Z))

#Prediction
XX = [[0, 0, 0, 0]]
a0 = gam.coef_[-1]
a1 = gam.partial_dependence(term=0, X=XX)
a2 = gam.partial_dependence(term=1, X=XX)
a3 = gam.partial_dependence(term=2, X=XX)
np.exp(a1+a2+a3+a0)
gam.predict(XX)

plt.scatter(gam.predict(X),y)
plt.plot(y,y,'r--')

###
X, y = mcycle(return_X_y=True)
gam = LinearGAM(n_splines=25).gridsearch(X, y)
XX = gam.generate_X_grid(term=0, n=500)

plt.plot(XX, gam.predict(XX), 'r--')
plt.plot(XX, gam.prediction_intervals(XX, width=.95), color='b', ls='--')

plt.scatter(X, y, facecolor='gray', edgecolors='none')
plt.title('95% prediction interval')

###
X, y = faithful(return_X_y=True)
gam = PoissonGAM().gridsearch(X, y)

plt.hist(faithful(return_X_y=False)['eruptions'], bins=200, color='k');
plt.plot(X, gam.predict(X), color='r')
